In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
import tensorflow as tf
#Dense
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        self.units = units
        super(MyDense, self).__init__(**kwargs)

    #build方法一般定义Layer需要被训练的参数。    
    def build(self, input_shape): 
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='w')
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='b')
        super(MyDense,self).build(input_shape) # 相当于设置self.built = True

    #call方法一般定义正向传播运算逻辑，__call__方法调用了它。    
    def call(self, inputs): 
        return tf.matmul(inputs, self.w) + self.b

    #如果要让自定义的Layer通过Functional API 组合成模型时可以序列化，需要自定义get_config方法。
    def get_config(self):  
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})
        return config

In [4]:
from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
labels = iris.target
# from sklearn.preprocessing import MinMaxScaler
# data=MinMaxScaler().fit_transform(data)

In [5]:
data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
labels#(150,3)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
#网络   函数式构建的网络
inputs = tf.keras.Input(shape=(4,))  
x = MyDense(units=16)(inputs) 
x = tf.nn.tanh(x) 
x = MyDense(units=3)(x) #0,1,2
# x= tf.keras.layers.Dense(16)(x)
predictions = tf.nn.softmax(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)


In [8]:
#shuffle:

In [9]:
import numpy as np

In [10]:
data = np.concatenate((data,labels.reshape(150,1)),axis=-1)
np.random.shuffle(data)

In [11]:
labels = data[:,-1]
data = data[:,:4]

In [12]:
#labels  ==[1,0,0]

In [13]:
#优化器 Adam
#损失函数 交叉熵损失函数
#评估函数 #acc


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

#keras
model.fit(data, labels, batch_size=32, epochs=100,shuffle=True)

Epoch 1/100
5/5 [==============================] - 0s 825us/step - loss: 1.1036 - sparse_categorical_accuracy: 0.3429
Epoch 2/100
5/5 [==============================] - 0s 742us/step - loss: 1.1022 - sparse_categorical_accuracy: 0.3329
Epoch 3/100
5/5 [==============================] - 0s 721us/step - loss: 1.0983 - sparse_categorical_accuracy: 0.4787
Epoch 4/100
5/5 [==============================] - 0s 666us/step - loss: 1.0969 - sparse_categorical_accuracy: 0.6584
Epoch 5/100
5/5 [==============================] - 0s 688us/step - loss: 1.0929 - sparse_categorical_accuracy: 0.4764

In [14]:
# _custom_objects = {
#     "Mylayer" :  Line,
# }

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
my_dense (MyDense)           (None, 16)                80        
_________________________________________________________________
tf.math.tanh (TFOpLambda)    (None, 16)                0         
_________________________________________________________________
my_dense_1 (MyDense)         (None, 3)                 51        
_________________________________________________________________
tf.nn.softmax (TFOpLambda)   (None, 3)                 0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.save('keras_model_tf_version.h5')

In [17]:
_custom_objects = {
    "MyDense" :  MyDense,
    
}

In [18]:
new_model = tf.keras.models.load_model("keras_model_tf_version.h5",custom_objects=_custom_objects)

In [19]:
y_pred = new_model.predict(data)

In [20]:
np.argmax(y_pred,axis=1)

array([1, 2, 2, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 2, 1, 1, 1, 2, 2,
       0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 1, 2, 0, 0, 2, 1,
       0, 1, 2, 0, 1, 1, 0, 2, 1, 0, 1, 2, 1, 2, 0, 1, 0, 2, 1, 0, 2, 1,
       1, 2, 2, 1, 2, 1, 2, 0, 1, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 2, 1, 2,
       0, 1, 0, 0, 2, 1, 2, 0, 1, 0, 1, 0, 2, 0, 0, 2, 0, 1, 2, 2, 0, 2,
       0, 1, 2, 2, 2, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 0, 2, 2, 1, 0, 2,
       2, 2, 2, 0, 1, 1, 2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 2, 0])

In [21]:
labels

array([1., 2., 2., 0., 2., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 2.,
       1., 1., 1., 2., 2., 0., 2., 2., 2., 2., 1., 0., 0., 2., 0., 1., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 2., 1., 0., 1., 2., 0., 1., 1., 0.,
       2., 1., 0., 1., 2., 1., 2., 0., 1., 0., 2., 1., 0., 2., 1., 1., 2.,
       2., 1., 1., 1., 2., 0., 1., 2., 0., 2., 1., 0., 1., 1., 0., 2., 1.,
       2., 1., 1., 0., 1., 0., 0., 2., 1., 2., 0., 1., 0., 1., 0., 2., 0.,
       0., 2., 0., 1., 2., 2., 0., 2., 0., 1., 2., 1., 2., 2., 2., 1., 2.,
       0., 1., 0., 2., 1., 2., 1., 0., 2., 2., 1., 0., 2., 2., 2., 2., 0.,
       1., 1., 2., 0., 1., 2., 0., 0., 2., 2., 0., 1., 2., 0.])